In [ ]:
def predictor(x_star_u, x_star_v,ModelInfo):
    
    x_b = ModelInfo['x_b']
    x_u = ModelInfo['x_u']
    x_v = ModelInfo['x_v']
    
    u_b = ModelInfo['u_b']
    u = ModelInfo['u']
    v = ModelInfo['v']
    
    y = np.concatenate((u_b, u, v))
    
    N_b = len(u_b)
    N_u = len(u)
    N_v = len(v)
    
    S0 = ModelInfo['S0']
    
    S = np.block([[np.zeros((N_b, N_b)), np.zeros((N_b, N_u + N_v))],
                  [np.zeros((N_u + N_v, N_b)), S0]])
    
    hyp = ModelInfo['hyp']
    
    K_nn_uu = k_nn_uu(x_star_u, x_b, hyp[:-2], 0,ModelInfo)
    K_nn1_uu = k_nn1_uu(x_star_u, x_u, hyp[:-2], 0,ModelInfo)
    K_nn1_uv = k_nn1_uv(x_star_u, x_v, hyp[:-2], 0,ModelInfo)
    
    K_nn_vu = k_nn_uv(x_b, x_star_v, hyp[:-2], 0,ModelInfo).T
    K_nn1_vu = k_nn1_vu(x_star_v, x_u, hyp[:-2], 0,ModelInfo)
    K_nn1_vv = k_nn1_vv(x_star_v, x_v, hyp[:-2], 0,ModelInfo)
    
    psi = np.block([[K_nn_uu, K_nn1_uu, K_nn1_uv],
                    [K_nn_vu, K_nn1_vu, K_nn1_vv]])
    
    V_nn_uu = k_nn_uu(x_star_u, x_star_u, hyp[:-2], 0,ModelInfo)
    V_nn_uv = k_nn_uv(x_star_u, x_star_v, hyp[:-2], 0,ModelInfo)
    V_vv_vv = k_nn_vv(x_star_v, x_star_v, hyp[:-2], 0,ModelInfo)
    
    VV = np.block([[V_nn_uu, V_nn_uv],
                   [V_nn_uv.T, V_vv_vv]])
    
    L = ModelInfo['L']
    
    f = np.dot(psi, np.linalg.solve(L.T, np.linalg.solve(L, y)))

    alpha = np.linalg.solve(L.T, np.linalg.solve(L, psi.T))

    v = VV - np.dot(psi, alpha) + np.dot(alpha.T, np.dot(S, alpha))
    
    return f, v
